In [1]:
#install packages
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
# Load each CSV file into a DataFrame
living_wages_df = pd.read_csv('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Term Project/housingproj/CIS-4400-Project-9/LivingWages/Csv/LivingWages3.csv', usecols=['state_id', 'minlivingwage', 'maxlivingwage', 'avglivingwage'])
dim_location_df = pd.read_csv('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Term Project/housingproj/CIS-4400-Project-9/Location/csv/dim_location1.csv', usecols=['state_id', 'location_id', 'county_name'])
housemarket_df = pd.read_csv('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Term Project/housingproj/CIS-4400-Project-9/HousingMarket/Scripts/csv/dim_housingmarket.csv', usecols=['state_id', 'housing_id'])
saleprice_df = pd.read_csv('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Term Project/housingproj/CIS-4400-Project-9/HousingMarket/Scripts/csv/housingsales2_df.csv', usecols=['state_id', 'sale_price'])

In [4]:
# Assuming the DataFrames are already loaded with all columns
# Now select specific columns
dim_location_df = dim_location_df[['state_id', 'location_id', 'county_name']]
housemarket_df = housemarket_df[['state_id', 'housing_id']]
saleprice_df = saleprice_df[['state_id', 'sale_price']]
living_wages_df = living_wages_df[['state_id', 'minlivingwage', 'maxlivingwage', 'avglivingwage']]

In [5]:
# Save dim_location_df to Parquet with selected columns
dim_location_df.to_parquet('dim_location_df.parquet', index=False, engine='pyarrow')

# Save housemarket_df to Parquet with selected columns
housemarket_df.to_parquet('housemarket_df.parquet', index=False, engine='pyarrow')

# Save saleprice_df to Parquet with selected columns
saleprice_df.to_parquet('saleprice_df.parquet', index=False, engine='pyarrow')

# Save living_wages_df to Parquet with selected columns
living_wages_df.to_parquet('living_wages_df.parquet', index=False, engine='pyarrow')


In [7]:
test_df = pd.read_parquet('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Term Project/housingproj/CIS-4400-Project-9/HousingMarket/Scripts/parquet/dim_location_df.parquet', engine='pyarrow')
test_df.head()

,state_id,location_id,county_name
0,CA,1,Los Angeles
1,CA,2,Los Angeles
2,CA,3,Los Angeles
3,CA,4,Los Angeles
4,CA,5,Los Angeles


In [8]:
dim_location_df = pd.read_parquet('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Term Project/housingproj/CIS-4400-Project-9/HousingMarket/Scripts/parquet/dim_location_df.parquet', engine='pyarrow')
housemarket_df = pd.read_parquet('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Term Project/housingproj/CIS-4400-Project-9/HousingMarket/Scripts/parquet/housemarket_df.parquet', engine='pyarrow')
saleprice_df = pd.read_parquet('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Term Project/housingproj/CIS-4400-Project-9/HousingMarket/Scripts/parquet/saleprice_df.parquet', engine='pyarrow')
living_wages_df = pd.read_parquet('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Term Project/housingproj/CIS-4400-Project-9/HousingMarket/Scripts/parquet/living_wages_df.parquet', engine='pyarrow')


In [ ]:
# Merge DataFrames step by step
merged_df1 = pd.merge(dim_location_df, housemarket_df, on='state_id', how='inner')
merged_df2 = pd.merge(merged_df1, saleprice_df, on='state_id', how='inner')
realestate_facts = pd.merge(merged_df2, living_wages_df, on='state_id', how='inner')

# Add a 'fact_id' column starting from 1 and incrementing by 1 for each row
realestate_facts.reset_index(drop=True, inplace=True)
realestate_facts.insert(0, 'fact_id', range(1, len(realestate_facts) + 1))

# Check the final DataFrame
realestate_facts.head()